In [1]:
!pip install transformers torch datasets transformers[torch]

In [2]:
from datasets import load_dataset

# Load the SQuAD dataset
squad = load_dataset("squad")

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [3]:
from transformers import LukeTokenizer, LukeForQuestionAnswering, RobertaTokenizerFast
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "tpu")

spans_tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base-lite")
model = LukeForQuestionAnswering.from_pretrained("studio-ousia/luke-base-lite")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/589M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of LukeForQuestionAnswering were not initialized from the model checkpoint at studio-ousia/luke-base-lite and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# i am too broke for this
train_size = len(squad["train"]) // 4
validation_size = len(squad["validation"]) // 4

squad["train"] = squad["train"].select(range(train_size))
squad["validation"] = squad["validation"].select(range(validation_size))

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]
    inputs = spans_tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []


    for i, offset in enumerate(offset_mapping):
        try:
            answer = answers[i]
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            context_start = sequence_ids.index(1)
            context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

            # If the answer is not fully inside the context, label it (0, 0)
            if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                start_positions.append(next((idx for idx, val in enumerate(offset) if val[0] <= start_char < val[1]), 0))
                end_positions.append(next((idx for idx, val in enumerate(offset) if val[0] < end_char <= val[1]), 0))
        except Exception as e:
            # Skip the problematic example
            print(f"Skipping example {i} due to error: {e}")
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/21899 [00:00<?, ? examples/s]

Map:   0%|          | 0/2642 [00:00<?, ? examples/s]

In [6]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [8]:
import numpy as np
import evaluate
from transformers import EvalPrediction
metric = evaluate.load("squad")

def compute_metrics(p: EvalPrediction):
    print(p)
    print(p[0])
    print(p[1])
    print(p[2])
    print(p[3])
    print(p[4])
    return metric.compute(predictions=p[0], references=p[1])

In [9]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

KeyboardInterrupt



In [ ]:
results = trainer.evaluate()

print(results)

In [10]:
from datasets import load_metric

In [ ]:
def get_validation_accuracy(trainer, tokenized_squad):
    # Get predictions
    predictions = trainer.predict(tokenized_squad["validation"])
    # Compute metrics
    metrics = compute_metrics(predictions)
    return metrics

# Evaluate the model and get validation accuracy
validation_metrics = get_validation_accuracy(trainer, tokenized_squad)
print(validation_metrics)
prin

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()


In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

In [ ]:
evaluations